In [971]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import math
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow as tf

In [972]:
# Load the dataset
data = pd.read_csv("BTC-USD.csv")

df = pd.DataFrame(data[0:-1])

df["Close"]

0      27129.585938
1      26753.826172
2      26851.277344
3      27225.726563
4      26334.818359
           ...     
361    66267.492188
362    65231.582031
363    67051.875000
364    66940.804688
365    66278.367188
Name: Close, Length: 366, dtype: float64

In [973]:
# Prepare Data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df["Close"].values.reshape(-1, 1))

scaled_data

array([[4.18048230e-02],
       [3.39697734e-02],
       [3.60017492e-02],
       [4.38094723e-02],
       [2.52329493e-02],
       [2.81810756e-02],
       [3.32496733e-02],
       [3.63578083e-02],
       [6.17398519e-02],
       [5.46554055e-02],
       [5.37476444e-02],
       [4.36829396e-02],
       [3.53490083e-02],
       [4.43070508e-02],
       [4.06693274e-02],
       [4.15854779e-02],
       [1.32493214e-02],
       [4.40817192e-02],
       [2.54660594e-02],
       [2.88485183e-02],
       [2.82679828e-02],
       [1.51497553e-02],
       [1.70040071e-02],
       [1.62185840e-02],
       [1.65569684e-02],
       [0.00000000e+00],
       [9.41888708e-03],
       [2.50795788e-02],
       [2.88997911e-02],
       [2.52620270e-02],
       [3.59965771e-02],
       [6.67825484e-02],
       [1.02225632e-01],
       [9.98274154e-02],
       [1.16157830e-01],
       [1.13097425e-01],
       [1.11670501e-01],
       [1.07309868e-01],
       [1.16005519e-01],
       [1.03454795e-01],


In [974]:
prediction_days = 300

x_train = []
y_train = []

for x in range(prediction_days, len(scaled_data)):
    x_train.append(scaled_data[x-prediction_days:x, 0])
    y_train.append(scaled_data[x, 0])

x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

In [975]:
# Defining stochastic Activation Function

class StochasticActivation(tf.keras.layers.Layer):
    def __init__(self, gamma, reaction_func, noise_func, activation="relu", **kwargs):
        super(StochasticActivation, self).__init__(**kwargs)
        self.gamma = gamma
        self.reaction_func = reaction_func
        self.activation = tf.keras.activations.get(activation)  # Get activation function by name
        self.noise_func = noise_func  # Define noise_func here (replace with your noise function definition)

    def call(self, inputs, prev_state=None):
        # Apply the formula
        if prev_state is None:
            prev_state = tf.zeros_like(inputs)  # Initialize for first state
        ht = self.activation(inputs)  # Replace 'activation' with your base activation function (e.g., tf.nn.relu)
        xi_t = self.noise_func(tf.shape(inputs))  # Sample random variable
        st = ht + self.gamma * xi_t * self.reaction_func(ht, prev_state)
        return st  # Return current and updated state

In [976]:
# Define Noise Function (Example: Gaussian Noise)
def noise_func(shape):
  return tf.random.normal(shape=shape, mean=0.0, stddev=0.6)  # Adjust stddev for noise level

In [977]:
# Define Reaction Function (Example: Identity Function)
def reaction_func(ht, prev_state):
  return ht - prev_state  # Adjust this function as needed (e.g., scaling factor)

In [978]:
# Building the Model
model = Sequential()
model.add(Dense(100, activation="relu", input_shape=(prediction_days, 1)))
model.add(StochasticActivation(gamma=0.8, reaction_func=reaction_func, noise_func=noise_func))  # Adjust gamma for noise impact
model.add(Dense(1, activation='linear'))

d:\My Drive\University\Masters\Deep Neural Network\Presentation\Implementation\venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [979]:
# Compile the Model
model.compile(optimizer="adam", loss="mse")

In [980]:
# Train the Model
model.fit(x_train, y_train, epochs=120, batch_size=32)

Epoch 1/120
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.7951
Epoch 2/120
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.7379 
Epoch 3/120
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.7001 
Epoch 4/120
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6578 
Epoch 5/120
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.6146 
Epoch 6/120
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5733 
Epoch 7/120
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.5424 
Epoch 8/120
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.5058 
Epoch 9/120
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.4738 
Epoch 10/120
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.4434 
Epoch 11/120
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.4152 
Epoch 12/120
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.3813 
Epoch 13/120
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.3619 
Epoch 14/120
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.3326 
Epoch 15/120
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.3054 
Epoch 16/120
3/3 ━━━

In [981]:
# Make Predictions (assuming you have scaled_data beyond prediction_days)
future_price = len(scaled_data)
predicted_price = model.predict(scaled_data[future_price-prediction_days:future_price, 0].reshape(1, prediction_days, 1))

predicted_price

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step


array([[[0.9166026 ],
        [0.80497336],
        [0.9218401 ],
        [0.8713299 ],
        [0.81265295],
        [0.8079175 ],
        [0.7323549 ],
        [0.8358716 ],
        [0.7509701 ],
        [0.77177405],
        [0.807835  ],
        [0.71399677],
        [0.8398378 ],
        [0.7395456 ],
        [0.8748523 ],
        [0.85536695],
        [0.7916601 ],
        [0.90881085],
        [0.69418776],
        [0.781417  ],
        [0.800733  ],
        [0.7481723 ],
        [0.86967194],
        [0.7296351 ],
        [0.7841704 ],
        [0.779137  ],
        [0.693663  ],
        [0.73385525],
        [0.8410499 ],
        [0.7211218 ],
        [0.78337204],
        [0.8670794 ],
        [0.76427853],
        [0.6964215 ],
        [0.8365773 ],
        [0.77783275],
        [0.8036063 ],
        [0.8339975 ],
        [0.76809597],
        [0.7999587 ],
        [0.71845734],
        [0.95868564],
        [0.80994725],
        [0.7853428 ],
        [0.7494128 ],
        [0

In [982]:
predictions = []

for i in range(100):
    # Invert Scaling for Actual Price Prediction
    sample = scaled_data[future_price-prediction_days:future_price, 0].reshape(1, prediction_days, 1)
    prediction = model.predict(sample)[0, 0]  # Access the first element from the first prediction
    predicted_price = scaler.inverse_transform(np.array([prediction]))  # Wrap the prediction in a NumPy array
    predictions.append(predicted_price[0][0])

mean_prediction = np.mean(predictions)
print(f"Predicted Price for Day {future_price}: ${mean_prediction:.2f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━

In [983]:
target_date = "2024-05-20"

# Find the row corresponding to the target date
target_row = data[data["Date"] == target_date]

# Extract the actual closing price for that day
actual_price = target_row["Close"].values[0]

print(f"Actual Price for Day {future_price}: ${actual_price:.2f}")

Actual Price for Day 366: $67282.47


In [986]:
# Calculate the squared differences between actual and predicted prices
squared_errors = (actual_price - predicted_price[0][0]) ** 2

# Compute the mean squared error
mse = np.mean(squared_errors)

print(f"Mean Squared Error: {mse:.2f}")

Mean Squared Error: 32136276.63
